In [ ]:
import datetime as dt
import logging
import json
from pathlib import Path
from s3pathlib import S3Path
from pyarrow import fs

import pandas as pd

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

## Setup data loading, local or s3 bucket

In [ ]:
# Setup data stream, either local filesys or s3
dataset_name = "state_level_simplified"

data_dir = Path("/datasets/dsgrid/dsgrid-tempo-v2022/")
use_s3 = False

if not data_dir.exists():
    use_s3 = True
    data_dir = S3Path("nrel-pds-dsgrid", "tempo/tempo-2022/v1.0.0/")
    s3 = fs.S3FileSystem(region=fs.resolve_s3_region('nrel-pds-dsgrid'))
    
# helper open fn:
def open_handler(file_path):
    if use_s3:
        return s3.open_input_stream('/'.join((file_path.bucket, file_path.key)))
    else:
        return open(file_path)


## Partitioned File Utilities

In [ ]:
def is_partitioned(filepath):
    for p in filepath.iterdir():
        if p.is_dir() and ("=" in p.stem) and (len(p.stem.split("=")) == 2):
            return True
    return False

def get_partitions(filepath):
    assert is_partitioned(filepath), f"{filepath} is not partitioned"
    
    partition_name = None
    for p in filepath.iterdir():
        if p.is_dir() and ("=" in p.stem):
            tmp, value = p.stem.split("=")
            if partition_name:
                assert (tmp == partition_name), f"Found two different partition names in {filepath}: {partition_name}, {tmp}"
            partition_name = tmp
            yield partition_name, value, p

def print_partitions(filepath, print_depth=2, _depth=0):
    if is_partitioned(filepath):
        space = ' ' * 4 * _depth
        for partition_name, value, p in get_partitions(filepath):
            print(f"{space}{partition_name}={value}")
        if (not print_depth) or ((_depth + 1) < print_depth):
            print_partitions(p, print_depth=print_depth, _depth=_depth+1)

## Load Data

In [ ]:
def get_metadata(dataset_path):
    with open_handler(dataset_path / "metadata.json") as f:
        result = json.load(f)
    return result

# load metadata and get column names by type
metadata = get_metadata(data_dir / dataset_name)
assert metadata["table_format"]["format_type"] == "unpivoted", metadata["table_format"]
value_column = metadata["table_format"]["value_column"]
columns_by_type = {dim_type: metadata["dimensions"][dim_type][0]["column_names"][0] 
                   for dim_type in metadata["dimensions"] if metadata["dimensions"][dim_type]}

In [ ]:
# Load data table
filepath = data_dir / dataset_name / "table.parquet"

filestr = filepath.uri if use_s3 else str(filepath)

df = pd.read_parquet(filestr)
    
logger.info(f"df.dtypes = \n{df.dtypes}")
df.head(5)


In [ ]:
print(f"Dataset contains {len(df.index):,} data points")

## Recreate Lefthand Side of Figure ES-1¶

In [ ]:
df2 = (df.groupby(["scenario", columns_by_type["model_year"]])[value_column].sum() / 1.0E6).reset_index()
df2.rename({columns_by_type["model_year"]: "year", value_column: "annual_twh"}, axis=1, inplace=True)
df2["scenario"] = df2["scenario"].map({
    "efs_high_ldv": "EFS High Electrification",
    "ldv_sales_evs_2035": "All LDV Sales EV by 2035",
    "reference": "AEO Reference"
})

In [ ]:
import plotly.express as px

fig = px.line(df2, x="year", y="annual_twh", color="scenario", 
              labels={"annual_twh": "EV Load (TWh/yr)", "scenario": "Scenario"}, 
              range_y=[-25,1025],
              width=600, height=450, template="plotly_white")
fig

## Verify Timestamps Are As Expected

In [ ]:
assert columns_by_type["time"] == "time_est", "Code in this section only makes sense if the dataset has timestamps"

In [ ]:
# select a subset of the data and look at initial timestamps

inds = (df["scenario"] == 'reference') & (df[columns_by_type['model_year']] == '2050')

if columns_by_type['geography'] == "census_division":
    inds &= (df[columns_by_type['geography']] == 'middle_atlantic')
elif columns_by_type['geography'] == "state":
    inds &= (df[columns_by_type['geography']] == 'RI')
elif columns_by_type['geography'] == "county":
    inds &= (df[columns_by_type['geography']] == '39023')
else:
    raise NotImplementedError()

if "subsector" not in columns_by_type:
    pass
elif columns_by_type['subsector'] == "subsector":
    inds &= (df[columns_by_type['subsector']] == 'bev_compact')
elif columns_by_type['subsector'] == "household_and_vehicle_type":
    inds &= (df[columns_by_type['subsector']] == 'Some_Drivers_Larger+Low_Income+Second_City+Pickup+BEV_100')
else:
    raise NotImplementedError()

if columns_by_type['metric'] == "end_uses_by_fuel_type":
    pass
elif columns_by_type['metric'] == "end_use":
    inds &= (df[columns_by_type['metric']] == 'electricity_ev_l1l2')
else:
    raise NotImplementedError()

df2 = df[inds].sort_values("time_est")
df2.head(5)

**When loading from .parquet, the first timestamp listed is "2012-01-01 05:00:00". Thus Pandas imports time_est as timezone-unaware timestamps in UTC in this case.**

**When loading from .csv with kwarg parse_dates=["time_est"], the first timestamp listed is "2012-01-01 00:00:00" such that no further transformation is needed to show the time in EST.**

In [ ]:
# demonstrate how to shift timestamps to match 'time_est' label

df3 = df2.copy()
df3["time_est"] = df3["time_est"] - dt.timedelta(hours=5)
df3.head(5)

## Verify that Profiles in Different Timezones Are As Expected

In [ ]:
assert columns_by_type["time"] == "time_est", "Code in this section only makes sense if the dataset has timestamps"

In [ ]:
def get_profile(start_timestamp, end_timestamp, inds, value_column=value_column, 
                normalize_profile=True, replace_timestamps=True):
    
    inds &= ((df['time_est'] >= start_timestamp) & (df['time_est'] <= end_timestamp))
    df2 = df[inds].groupby("time_est")[value_column].sum().reset_index().sort_values("time_est")
    
    if normalize_profile:
        df2[value_column] = df2[value_column] / df2[value_column].sum()
    if replace_timestamps:
        df2["hour"] = df2.index.values
        df2 = df2[["hour",value_column]]
    return df2

inds = (df["scenario"] == 'reference') & (df[columns_by_type['model_year']] == '2050')

if "subsector" not in columns_by_type:
    pass
elif columns_by_type['subsector'] == "subsector":
    inds &= (df[columns_by_type['subsector']] == 'bev_compact')
elif columns_by_type['subsector'] == "household_and_vehicle_type":
    inds &= (df[columns_by_type['subsector']] == 'Some_Drivers_Smaller+Middle_Income+Suburban+SUV+BEV_300')
else:
    raise NotImplementedError()

geographies = None
if columns_by_type['geography'] == "census_division":
    geographies = {"ET": "middle_atlantic", "CT": "west_south_central", "MT": "mountain", "PT": "pacific"}
elif columns_by_type['geography'] == "state":
    geographies = {"ET": "NC", "CT": "TX", "MT": "CO", "PT": "OR"}
elif columns_by_type['geography'] == "county":
    geographies = {"ET": "37183", "CT": "48453", "MT": "08069", "PT": "06059"}
else:
    raise NotImplementedError()

days = {
    "Standard Time": (dt.datetime(2012, 2, 14, 5), dt.datetime(2012, 2, 15, 4)),        # Selects UTC timestamps that correspond to EST 2/14/2012
    "Daylight Savings Time": (dt.datetime(2012, 8, 14, 5), dt.datetime(2012, 8, 15, 4)) # Selects UTC timestamps that correspond to EST 8/14/2012
}

data = []
for time_type, time_tuple in days.items():
    for tz, geo in geographies.items():
        data.append(get_profile(time_tuple[0], time_tuple[1], inds & (df[columns_by_type['geography']] == geo)))
        data[-1]["Time Type"] = time_type
        data[-1]["Time Zone"] = tz
df2 = pd.concat(data)
df2

In [ ]:
import plotly.express as px

fig = px.line(df2, x="hour", y=value_column, color="Time Zone", line_dash="Time Type",
              color_discrete_map={"ET": "red", "CT": "orange", "MT": "blue", "PT": "purple"},
              labels={"value": "Normalized Load Profile", "hour": "Hour of EST Day"},
              #range_y=[0,0.1],
              width=600, template="plotly_white")
fig

## Demonstrate Loading a Subset of a Larger Dataset

In [ ]:
filepath = data_dir / dataset_name / "table.parquet"
print_partitions(filepath, print_depth=None)

In [ ]:
# Edit this list of tuples as desired
partitions=[
    ("scenario", "efs_high_ldv")
]

subset_filepath = filepath
for partition_name, value in partitions:
    subset_filepath = subset_filepath / f"{partition_name}={value}"

# load partial data table
df = pd.read_parquet(subset_filepath)
df.head(5)

In [ ]:
print(f"Dataset contains {len(df.index):,} data points")

## Loading CSVs

In [ ]:
# CSV's can be found in some of the aggregate summary datasets
filepath = data_dir / "annual_summary_state" / "table.csv"
filestr = filepath.uri if use_s3 else str(filepath)

kwargs = { 
    "dtype": { columns_by_type['model_year']: str }
}        
df = pd.read_csv(filestr, **kwargs)
df.head()